In [1]:
import tweepy
from econfig import *
import datetime as dt
import json
import psycopg2

In [21]:
conn = psycopg2.connect(user = "postgres",
                                  password = "dataisgreat",
                                  host = "127.0.0.1",
                                  port = "3306",
                                  database = "postgres")
cursor = conn.cursor()

In [22]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

try:
    api.verify_credentials()
    print("Authentication Complete")
except:
    print("Authentication Unable to Complete")
    
two_days = (dt.datetime.now() - dt.timedelta(days=8)).strftime("%Y-%m-%d")
one_day = (dt.datetime.now() - dt.timedelta(days=7)).strftime("%Y-%m-%d")

# startDate = datetime.datetime(2014, 6, 1, 0, 0, 0)
# endDate =   datetime.datetime(2015, 1, 1, 0, 0, 0)
MAX_TWEETS = 5
tweets = tweepy.Cursor(api.search, q='#election2020', rpp=100, since=two_days, until=one_day).items(MAX_TWEETS)

Authentication Complete


In [23]:

for tweet in tweets:
    followers = tweet.user.followers_count
    tweet_id = tweet.id
    user_id = tweet.user.id
    statuses = tweet.user.statuses_count
    entities = tweet.entities
    sn = tweet.user.screen_name
    date = tweet.created_at.strftime('%Y-%m-%d %H:%M:%S')
    text = tweet.text
    
#     CREATE TABLE new_tweets (date date, tweet_id bigint, user_id bigint, username text, statuses_count int, follower_count int, entities text, tweet_text text);
    cursor.execute("INSERT INTO new_tweets (date, tweet_id, user_id, username, statuses_count, follower_count, entities, tweet_text)\
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s);", (tweet.created_at.strftime('%Y-%m-%d %H:%M:%S'), tweet_id, user_id, sn, statuses, followers,json.dumps(entities), text))
    
    conn.commit();
    print("Tweet Added")


Tweet Added
Tweet Added
Tweet Added
Tweet Added
Tweet Added


In [24]:
cursor.execute("select * from new_tweets;")
fetch = cursor.fetchall()
print(fetch)

[(datetime.date(2020, 2, 22), 1231368047019122688, 17041374, 'wgbhnews', 81883, 31639, '{"hashtags": [], "symbols": [], "user_mentions": [], "urls": [{"url": "https://t.co/Tp4H2KyARe", "expanded_url": "https://twitter.com/i/web/status/1231368047019122688", "display_url": "twitter.com/i/web/status/1\\u2026", "indices": [117, 140]}]}', 'After being pummeled in his first debate, former NYC Mayor Mike Bloomberg told Boston business leaders that his nex… https://t.co/Tp4H2KyARe'), (datetime.date(2020, 2, 22), 1231368017889681408, 233374516, 'loveletters2007', 170515, 5053, '{"hashtags": [], "symbols": [], "user_mentions": [{"screen_name": "indictmentduck", "name": "Indictment Duck #TrumpIsOJ #WorkGiveVoteBlue", "id": 1090642487268524032, "id_str": "1090642487268524032", "indices": [3, 18]}, {"screen_name": "ewarren", "name": "Elizabeth Warren", "id": 357606935, "id_str": "357606935", "indices": [73, 81]}], "urls": []}', 'RT @indictmentduck: POLL TIME!!! Please Vote, Comment, &amp; Retweet\n

In [25]:
print(date)

2020-02-22 23:59:31


In [26]:
cursor.close()
conn.close()